In [1]:
import pandas as pd
from sklearn.cross_validation import KFold
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

/Users/tracesmith/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [40]:
data = pd.read_csv("./Data/train.csv",sep=",",header=0)

In [41]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,530,20,RL,0,32668,Pave,NAW,IR1,Lvl,AllPub,...,0,NP,NF,Nfeat,0,3,2007,WD,Alloca,200624
1,809,80,RL,85,13400,Pave,NAW,Reg,Lvl,AllPub,...,0,NP,GdWo,Nfeat,0,6,2006,WD,Normal,159950
2,1112,60,RL,80,10480,Pave,NAW,Reg,Lvl,AllPub,...,0,NP,NF,Nfeat,0,9,2008,WD,Normal,205000
3,38,20,RL,74,8532,Pave,NAW,Reg,Lvl,AllPub,...,0,NP,NF,Nfeat,0,10,2009,WD,Normal,153000
4,1269,50,RL,0,14100,Pave,NAW,IR1,Lvl,AllPub,...,0,NP,GdWo,Nfeat,0,5,2008,WD,Normal,381000


In [45]:
X_all = data.ix[:,:-1]
print("Explanatory Variables: {}".format(X_all.shape[1]))
y = data.ix[:,-1:]
print("Response Variable: {}".format(y.shape[1]))
print("Total Observations: {}".format(y.shape[0]))

Explanatory Variables: 80
Response Variable: 1
Total Observations: 1460


In [46]:
X_all.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,57.623288,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.117123,443.639726,...,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753
std,421.610009,42.300571,34.664304,9981.264932,1.382997,1.112799,30.202904,20.645407,180.731373,456.098091,...,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095
min,1.000000,20.000000,0.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,365.750000,20.000000,42.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000
50%,730.500000,50.000000,63.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,1095.250000,70.000000,79.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,164.250000,712.250000,...,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000


In [47]:
X_all.columns.values

array(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond',
       'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir',
       'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea',
       'GarageQual', 'GarageCond', 'Pav

In [48]:
for col,row in X_all.iteritems():
    print("{}: {} -- na: {}".format(col,row.dtype,row.isnull().sum()))

Id: int64 -- na: 0
MSSubClass: int64 -- na: 0
MSZoning: object -- na: 0
LotFrontage: int64 -- na: 0
LotArea: int64 -- na: 0
Street: object -- na: 0
Alley: object -- na: 0
LotShape: object -- na: 0
LandContour: object -- na: 0
Utilities: object -- na: 0
LotConfig: object -- na: 0
LandSlope: object -- na: 0
Neighborhood: object -- na: 0
Condition1: object -- na: 0
Condition2: object -- na: 0
BldgType: object -- na: 0
HouseStyle: object -- na: 0
OverallQual: int64 -- na: 0
OverallCond: int64 -- na: 0
YearBuilt: int64 -- na: 0
YearRemodAdd: int64 -- na: 0
RoofStyle: object -- na: 0
RoofMatl: object -- na: 0
Exterior1st: object -- na: 0
Exterior2nd: object -- na: 0
MasVnrType: object -- na: 0
MasVnrArea: int64 -- na: 0
ExterQual: object -- na: 0
ExterCond: object -- na: 0
Foundation: object -- na: 0
BsmtQual: object -- na: 0
BsmtCond: object -- na: 0
BsmtExposure: object -- na: 0
BsmtFinType1: object -- na: 0
BsmtFinSF1: int64 -- na: 0
BsmtFinType2: object -- na: 0
BsmtFinSF2: int64 -- na: 

### Set Explanatory Variables as "Categorical Variabes"

In [49]:
X_all['YearRemodAdd'] = X_all['YearRemodAdd'].astype(object)
X_all['YearBuilt'] = X_all['YearBuilt'].astype(object)
X_all['OverallCond'] = X_all['OverallCond'].astype(object)
X_all['OverallQual'] = X_all['OverallQual'].astype(object)
X_all['MSSubClass'] = X_all['MSSubClass'].astype(object)
#for col,row in X_all.iteritems():
    #print("{}: {} -- na: {}".format(col,row.dtype,row.isnull().sum()))

### Preprocess Data -- Dummy Variables

In [50]:
def preprocess_features(X):
    df_preprocess = pd.DataFrame(index=X.index) # output dataframe, initially empty
    # iterate through each column
    for colname, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object: #if data type == object (i.e. non-numeric)
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # If data is non-numeric and consist of more than 2 level
        # convert todummy variables (indicator variables)
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=colname)  
        #join columns to one dataframe
        df_preprocess = df_preprocess.join(col_data)  # collect column(s) in output dataframe
    return df_preprocess

In [51]:
X_all = data.ix[:,:-1] #get explanatory variables
y = data.ix[:,-1:] #get response variable
X_all = preprocess_features(X_all) # call the preprocess_features function
df_ = pd.concat([X_all,y],axis=1) #join the two df's
df_.head()# check everything looks correct

,Id,MSSubClass,MSZoning_C_all,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,LotFrontage,LotArea,Street_Grvl,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SalePrice
0,530,20,0.0,0.0,0.0,1.0,0.0,0,32668,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,200624
1,809,80,0.0,0.0,0.0,1.0,0.0,85,13400,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,159950
2,1112,60,0.0,0.0,0.0,1.0,0.0,80,10480,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,205000
3,38,20,0.0,0.0,0.0,1.0,0.0,74,8532,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,153000
4,1269,50,0.0,0.0,0.0,1.0,0.0,0,14100,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,381000


In [63]:
df_.columns.values

array(['Id', 'MSSubClass', 'MSZoning_C_all', 'MSZoning_FV', 'MSZoning_RH',
       'MSZoning_RL', 'MSZoning_RM', 'LotFrontage', 'LotArea',
       'Street_Grvl', 'Street_Pave', 'Alley_Grvl', 'Alley_NAW',
       'Alley_Pave', 'LotShape_IR1', 'LotShape_IR2', 'LotShape_IR3',
       'LotShape_Reg', 'LandContour_Bnk', 'LandContour_HLS',
       'LandContour_Low', 'LandContour_Lvl', 'Utilities_AllPub',
       'Utilities_NoSeWa', 'LotConfig_Corner', 'LotConfig_CulDSac',
       'LotConfig_FR2', 'LotConfig_FR3', 'LotConfig_Inside',
       'LandSlope_Gtl', 'LandSlope_Mod', 'LandSlope_Sev',
       'Neighborhood_Blmngtn', 'Neighborhood_Blueste',
       'Neighborhood_BrDale', 'Neighborhood_BrkSide',
       'Neighborhood_ClearCr', 'Neighborhood_CollgCr',
       'Neighborhood_Crawfor', 'Neighborhood_Edwards',
       'Neighborhood_Gilbert', 'Neighborhood_IDOTRR',
       'Neighborhood_MeadowV', 'Neighborhood_Mitchel',
       'Neighborhood_NAmes', 'Neighborhood_NPkVill', 'Neighborhood_NWAmes',
       'Neig

In [52]:
df_.to_csv("train_dummy.csv") #outout csv file

In [53]:
### Split Dataframe into 70% train 30% testing
from sklearn.model_selection import train_test_split
def test_train_split(X_all,y):
    X_train,X_test,y_train,y_test = train_test_split(X_all,y,test_size=0.30,random_state=42)
    return(X_train,X_test,y_train,y_test)

In [54]:
X_train,X_test,y_train, y_test=test_train_split(X_all,y)

In [55]:
#Training (70%)
training_ = pd.concat([X_train,y_train],axis=1)
training_.to_csv("train_split_dummy.csv")
print(training_.shape[0])

1022


In [56]:
#Testing (30%)
testing_ = pd.concat([X_test,y_test],axis=1)
#Add column "." for predictions in SAS
testing_['prediction'] = "."
testing_.to_csv("test_split_dummy.csv")

### Freature Importance (Random Forest)

In [57]:
#Feature Selection 
def feature_importance(reg,X_train,y_train):

    # Relative Importance (Features)
    reg.fit(X_train,y_train)
    
    # Get Feature Importance from the classifier
    feature_importance = reg.feature_importances_
    
    # Normalize The Features
    feature_importance = 100.0 * (feature_importance / feature_importance.max())

    # Sort Features and Creat Horizontal Bar Plot
    sorted_idx = np.argsort(feature_importance)
    importance = pd.DataFrame(sorted_idx,columns=['Importance'])
    feat_names= pd.DataFrame(np.asanyarray(X_train.columns.tolist())[sorted_idx],columns=['Features'])
    df = pd.concat([feat_names,importance],axis=1)
    df = df.sort(['Importance'], ascending=[0])
    print(df)

In [60]:
def run_model(reg,X,y):
    print("Model:", reg.__class__.__name__)
    print("*******************************")

    #initialize empty dataframe to store iteration results
    df_summary = pd.DataFrame(columns=['Fold','Run Time','Accuracy'])

    #Perform Cross-Validation
    n_folds = 5 # number of fold; (i.e. 3, 5, or 10 are common; odd numbers perfered)
    kf = KFold(len(X),n_folds,shuffle=True,random_state=None)

    #iterate through the indicies identified from kf and assign training and test 
    # for x and y, respectively. Use '.loc' for slicing dataframe
    for index,(train_index,test_index) in list(enumerate(kf,start=1)):
        print("Fold: ",index)
        X_train = X.loc[train_index]; X_test = X.loc[test_index]
        y_train = y.loc[train_index]; y_test = y.loc[test_index]
        
        df = pd.DataFrame(feature_importance(reg,X_train,y_train))
        print(df.head(20))
        print("==============================================")
    
if __name__ == "__main__":
    reg = RandomForestRegressor(n_estimators=1000)
    %time run_model(reg,X_all,y)

Model: RandomForestRegressor
*******************************
Fold:  1
                  Features  Importance
324  SaleCondition_Partial         400
332   SaleCondition_Normal         399
306   SaleCondition_Family         398
166   SaleCondition_Alloca         397
39   SaleCondition_AdjLand         396
294  SaleCondition_Abnorml         395
318            SaleType_WD         394
66            SaleType_Oth         393
335           SaleType_New         392
82          SaleType_ConLw         391
56          SaleType_ConLI         390
102         SaleType_ConLD         389
51            SaleType_Con         388
101           SaleType_CWD         387
202           SaleType_COD         386
367                 YrSold         385
381                 MoSold         384
219                MiscVal         383
10        MiscFeature_TenC         382
182       MiscFeature_Shed         381
45        MiscFeature_Othr         380
172      MiscFeature_Nfeat         379
26        MiscFeature_Gar2       